In [1]:
import inspect
import time
from statistics import mean, stdev
from CybORG import CybORG
from CybORG.Agents import B_lineAgent, SleepAgent, GreenAgent
from CybORG.Agents.SimpleAgents.BaseAgent import BaseAgent
from CybORG.Agents.SimpleAgents.BlueReactAgent import BlueReactRemoveAgent
from CybORG.Agents.SimpleAgents.Meander import RedMeanderAgent
from CybORG.Agents.Wrappers.EnumActionWrapper import EnumActionWrapper
from CybORG.Agents.Wrappers.FixedFlatWrapper import FixedFlatWrapper
from CybORG.Agents.Wrappers.OpenAIGymWrapper import OpenAIGymWrapper
from CybORG.Agents.Wrappers.ReduceActionSpaceWrapper import ReduceActionSpaceWrapper
from CybORG.Agents.Wrappers import ChallengeWrapper
import os

from ray.tune.registry import register_env
from CybORG.Agents.Wrappers.rllib_wrapper import RLlibWrapper
import warnings
import numpy as np
from ray import air, tune
warnings.filterwarnings('ignore')

In [2]:
def env_creator(env_config: dict):
    path = str(inspect.getfile(CybORG))
    path = path[:-10] + '/Shared/Scenarios/Scenario2.yaml'
    agents = {"Red": B_lineAgent, "Green": GreenAgent}
    cyborg = CybORG(scenario_file=path, environment='sim', agents=agents)
    env = RLlibWrapper(env=cyborg, agent_name="Blue", max_steps=100)
    return env

def print_results(results_dict):
    train_iter = results_dict["training_iteration"]
    r_mean = results_dict["episode_reward_mean"]
    r_max = results_dict["episode_reward_max"]
    r_min = results_dict["episode_reward_min"]
    print(f"{train_iter:4d} \tr_mean: {r_mean:.1f} \tr_max: {r_max:.1f} \tr_min: {r_min: .1f}")

register_env(name="CybORG", env_creator=env_creator)

In [ ]:

tune.Tuner(
        "PPO",
        run_config=air.RunConfig(
            stop={"timesteps_total": 10e6},
            local_dir='results/APPO', name="explore",
            checkpoint_config=air.CheckpointConfig(
                checkpoint_frequency=500, 
            ),
        ),
        param_space={
            # CC3 specific.
            "env": "CybORG",
            # General
            "num_gpus": 1,
            "num_workers": 30,
            "horizon": 100,
            "num_envs_per_worker": 2,
            #"exploration_config": tune.grid_search([{"type": "RE3",
            #    "embeds_dim": 128,
            #    "beta_schedule": "constant",
            #    "sub_exploration": {
            #        "type": "StochasticSampling",
            #},}, 
            "exploration_config": {"type": "StochasticSampling"},
            #algo params
            "train_batch_size": 2000,
            "lr": 0.000025,
            "gamma": 0.99,
            "framework": 'tf',
            "model": {
                    "fcnet_hiddens": [256, 256],
                    "fcnet_activation": "tanh",
                    'use_lstm': True,
                    'max_seq_len': 5,
                },
            
 
        },
    ).fit()

(PPO pid=11513) 2022-12-05 12:05:06,977	INFO algorithm.py:2303 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(PPO pid=11513) 2022-12-05 12:05:06,977	WARNING ppo.py:351 -- `train_batch_size` (2000) cannot be achieved with your other settings (num_workers=30 num_envs_per_worker=2 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 34.
(PPO pid=11513) 2022-12-05 12:05:06,977	INFO ppo.py:379 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(PPO pid=11513) 2022-12-05 12:05:06,979	INFO algorithm.py:457 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=11544) 2022-12-05 12:05:

Trial name,agent_timesteps_total,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_recreated_workers,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
PPO_CybORG_0b1b8_00000,5.06532e+06,"{'num_env_steps_sampled': 5065320, 'num_env_steps_trained': 5065320, 'num_agent_steps_sampled': 5065320, 'num_agent_steps_trained': 5065320}",{},2022-12-05_17-03-22,False,100,{},-12.2,-91.194,-350.8,60,50640,ba2306176fe64abcb3c08f0f6a7fee2d,01589170c3ff,"{'learner': {'default_policy': {'learner_stats': {'cur_kl_coeff': 0.20000000298023224, 'cur_lr': 2.499999936844688e-05, 'total_loss': 5.9119267, 'policy_loss': -0.025190657, 'vf_loss': 5.935811, 'vf_explained_var': -0.66114366, 'kl': 0.0065337783, 'entropy': 2.9006612, 'entropy_coeff': 0.0, 'model': {}}, 'train': None}}, 'num_env_steps_sampled': 5065320, 'num_env_steps_trained': 5065320, 'num_agent_steps_sampled': 5065320, 'num_agent_steps_trained': 5065320}",2483,172.28.0.2,5.06532e+06,5.06532e+06,5.06532e+06,2040,5.06532e+06,2040,0,30,0,2040,"{'cpu_util_percent': 25.991666666666664, 'ram_util_percent': 23.3}",11513,{},{},{},"{'mean_raw_obs_processing_ms': 4.721087315526949, 'mean_inference_ms': 3.333378075530862, 'mean_action_processing_ms': 0.21744370135640495, 'mean_env_wait_ms': 23.309351779254158, 'mean_env_render_ms': 0.0}","{'episode_reward_max': -12.199999999999978, 'episode_reward_min': -350.80000000000047, 'episode_reward_mean': -91.19399999999996, 'episode_len_mean': 100.0, 'episode_media': {}, 'episodes_this_iter': 60, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-56.80000000000009, -231.7, -283.79999999999984, -284.6999999999999, -19.299999999999994, -212.2, -18.799999999999986, -17.700000000000006, -21.800000000000026, -145.59999999999988, -57.80000000000009, -27.600000000000033, -58.60000000000007, -20.700000000000003, -199.79999999999978, -18.799999999999994, -13.199999999999974, -265.7999999999997, -109.60000000000005, -16.799999999999986, -38.60000000000004, -197.79999999999978, -12.199999999999978, -70.60000000000005, -217.79999999999976, -269.7999999999998, -56.80000000000009, -273.7999999999998, -15.699999999999978, -18.7, -22.700000000000017, -142.60000000000002, -37.600000000000094, -57.80000000000009, -14.79999999999998, -271.7999999999998, -271.6999999999998, -253.79999999999967, -58.8000000000001, -89.80000000000004, -282.6999999999999, -92.79999999999995, -57.80000000000009, -233.79999999999973, -97.80000000000005, -12.799999999999976, -21.400000000000013, -18.80000000000002, -217.79999999999976, -15.799999999999976, -14.399999999999983, -16.59999999999999, -228.79999999999973, -97.79999999999995, -81.59999999999997, -350.80000000000047, -57.80000000000009, -19.700000000000003, -15.599999999999973, -19.800000000000004, -56.80000000000009, -57.80000000000009, -69.20000000000003, -14.79999999999997, -38.40000000000007, -117.79999999999991, -287.80000000000007, -57.70000000000009, -57.80000000000009, -18.7, -19.90000000000002, -18.800000000000004, -20.400000000000023, -56.70000000000009, -62.400000000000034, -12.499999999999979, -12.699999999999989, -18.80000000000001, -19.800000000000022, -16.799999999999986, -56.80000000000009, -18.399999999999988, -48.700000000000074, -24.70000000000001, -17.099999999999998, -57.70000000000009, -211.79999999999976, -46.80000000000011, -64.70000000000007, -19.700000000000014

In [5]:
!nvidia-smi

Mon Dec  5 12:03:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.03   Driver Version: 450.119.03   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:83:00.0 Off |                    0 |
| N/A   39C    P0    38W / 250W |   2167MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip show ray

In [ ]:
path = str(inspect.getfile(CybORG))
path = path[:-10] + '/Shared/Scenarios/Scenario2.yaml'
agents = {"Red": B_lineAgent, "Green": GreenAgent}
cyborg = CybORG(scenario_file=path, environment='sim', agents=agents)
env = RLlibWrapper(env=cyborg, agent_name="Blue", max_steps=100)

In [ ]:
env.observation_space

In [18]:
dist = np.array([0.9997, 0.0001, 0.0001, 0.0001])
entropy = -np.sum(dist * np.log(dist))
entropy

0.0030630571070921465

In [19]:
dist = np.array([0.5, 0.5, 0.0001, 0.0001])
entropy = -np.sum(dist * np.log(dist))
entropy

0.6949892486343404

In [20]:
dist = np.array([0.25, 0.25, 0.25, 0.25])
entropy = -np.sum(dist * np.log(dist))
entropy

1.3862943611198906